# Import

In [14]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from nltk import word_tokenize, download
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import matplotlib.pyplot as plt

## Add configuration file

In [15]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [16]:
from ALL import config 
from util import *

## Set condition

In [17]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [18]:
s3 = S3Manager()

In [19]:
data_type = "20News"
sentence_counts = [2, 4, 8, 16, 32, 64, 128]

# Read data

In [20]:
df_paths = {
    sentence_count: s3.download(
        f"Preprocessing/{data_type}Sampled{sentence_count}/master.csv"
    )[0]
    for sentence_count in sentence_counts
}

In [21]:
dfs = {sentence_count: pd.read_csv(df_path, index_col=0) for sentence_count, df_path in df_paths.items()}

In [22]:
labels_paths = {
    sentence_count: s3.download(
        f"Preprocessing/{data_type}Sampled{sentence_count}/class.csv"
    )[0]
    for sentence_count in sentence_counts
}

In [23]:
class_labels = {}
for sentence_count, labels_path in labels_paths.items():
    with open(labels_path, mode="r") as f:
        reader = csv.reader(f)
        class_labels[sentence_count] = [label for label in reader]

# Split texts

In [27]:
word_statistics = {
    sentence_count: dfs[sentence_count]["words_length"].describe()
    for sentence_count in sentence_counts
}

In [30]:
pd.concat(word_statistics, axis=1)

,2,4,8,16,32,64,128
count,18770.000000,18770.000000,18770.000000,18694.000000,18693.000000,18694.000000,18694.000000
mean,51.570538,98.614172,171.392488,242.818123,289.045579,313.568257,322.731839
std,32.785106,45.578389,69.260900,118.734389,190.421324,260.053213,303.003753
min,4.000000,16.000000,17.000000,17.000000,17.000000,17.000000,17.000000
25%,29.000000,67.000000,125.000000,148.000000,148.000000,148.000000,148.000000
50%,43.000000,90.000000,164.000000,232.000000,237.000000,237.000000,237.000000
75%,65.000000,120.000000,209.000000,325.000000,380.000000,381.000000,381.000000
max,288.000000,444.000000,617.000000,656.000000,1009.000000,1614.000000,2482.000000


# Visualize

## Describe

In [26]:
for data_type in data_types:
    print(data_type)
    print(dfs[data_type]["sentence_count"].describe())

NameError: name 'data_types' is not defined

In [ ]:
dfs["20News"]["sentence_count"].quantile(0.99)

In [ ]:
dfs["20News"]["sentence_count"]

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(dfs["20News"]["sentence_count"], bins=100)
plt.show()

In [ ]:
shutil.rmtree("../../nltk_data/")